### **Comparing range of observations and SMRT simulations with no atmosphere for the three main topography types**

In [ ]:
import pandas as pd
import xarray as xr
import numpy as np
%matplotlib widget
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import matplotlib.colors as colors
from matplotlib.ticker import AutoMinorLocator
from mpl_toolkits.axes_grid1.inset_locator import inset_axes

In [ ]:
%load_ext autoreload
%aimport aesop
%autoreload 1

Load observations:

In [ ]:
obs_c087 = aesop.get_obs_per_aoi('C087')
obs_c090 = aesop.get_obs_per_aoi('C090')

Generate new results with surface snow

In [ ]:
# UNCOMMENT TO RUN SIMULATIONS

# from run_smrt_with_realistic_atmosphere import run_smrt_with_atmosphere, tb_results
# res_with_SS = run_smrt_with_atmosphere('C090', extremes=True, fresh_snow=40)
# tb_results(res_with_SS, save=True, filename='../Other/smrt_results_atmos/smrt_atmos_tb_C090_with_SS.nc')

Load SMRT results with atmosphere:

In [ ]:
smrt_c087 = xr.open_dataset('../Other/smrt_results_atmos/smrt_atmos_tb_C087.nc')
smrt_c090 = xr.open_dataset('../Other/smrt_results_atmos/smrt_atmos_tb_C090.nc')
smrt_c090_SS = xr.open_dataset('../Other/smrt_results_atmos/smrt_atmos_tb_C090_with_SS.nc')

Load pits with topography:

In [ ]:
pits = aesop.get_pit_topography()

In [ ]:
def set_title(ax, channel):
    if channel in ['M16-89', 'M17-157']:
        ax.set_title(channel[4:]+' GHz')
    elif channel == '118+5.0':
        ax.set_title('118$\pm$5.0 GHz')
    elif channel == 'M20-183+7':
        ax.set_title('183$\pm$7 GHz')
    elif channel == '243':
        ax.set_title('243 GHz')

In [ ]:
def group_topography(flight_obs, smrt):
    sims = {}
    obs = {}
    for (topog, group) in pits.groupby('topo_type'):
        topo_pits = group.index
        grouped_obs = {}
        grouped_sims = {}

        for channel in smrt_c087.channel.values:
            grouped_obs.update({channel: 
                                flight_obs[channel][flight_obs[channel].topo_type==topog].TB.values})
            grouped_sims.update({channel:
                               [stat(smrt.sel(snowpack=topo_pits, channel=channel)[sim].values)
                                for stat, sim in zip([np.nanmean, np.nanmax, np.nanmin],
                                                     ['base','high','low'])]})
        obs.update({topog: grouped_obs})
        sims.update({topog: grouped_sims})    
    return obs, sims

In [ ]:
plt.rcParams.update({'font.size': 14})

fig, axes = plt.subplots(2,5, figsize=(16,8), sharey=True)

for channel, ax in zip(['M16-89','118+5.0','M17-157','M20-183+7','243'], axes[0,:]):
    
    positions = range(0,3)

    obs_87, sims_87 = group_topography(obs_c087, smrt_c087)
    o = [obs_87.get(topo).get(channel) for topo in obs_87.keys()]
    ax.boxplot(o, positions=positions, showfliers=False)

    base = np.array([sims_87.get(topo).get(channel)[0] for topo in sims_87.keys()])
    high = np.array([sims_87.get(topo).get(channel)[1] for topo in sims_87.keys()])
    low = np.array([sims_87.get(topo).get(channel)[2] for topo in sims_87.keys()])

    ax.errorbar(sims_87.keys(), base, yerr=[base-low, high-base],fmt='H',
                color='b', label='SMRT Sims', capsize=0, markeredgecolor='k',ecolor='dodgerblue',
                elinewidth=3)

    ax.set_xticklabels(obs_87.keys())
    set_title(ax, channel)
    ax.set_ylim([120,260])

axes[0,0].set_ylabel('TB (K) C087')

for channel, ax in zip(['M16-89','118+5.0','M17-157','M20-183+7','243'], axes[1,:]):
    
    positions = range(0,3)

    obs_90, sims_90SS = group_topography(obs_c090, smrt_c090_SS)
    o = [obs_90.get(topo).get(channel) for topo in obs_90.keys()]

    ax.boxplot(o, positions=positions, showfliers=False)

    base = np.array([sims_90SS.get(topo).get(channel)[0] for topo in sims_90SS.keys()])
    high = np.array([sims_90SS.get(topo).get(channel)[1] for topo in sims_90SS.keys()])
    low = np.array([sims_90SS.get(topo).get(channel)[2] for topo in sims_90SS.keys()])

    ax.errorbar(sims_90SS.keys(), base, yerr=[base-low, high-base],fmt='H',
                color='b', label='SMRT Sims', capsize=0, markeredgecolor='k',ecolor='dodgerblue',
                elinewidth=3)

    ax.set_xticklabels(obs_90.keys())
    ax.set_ylim([120,260])
    
axes[1,0].set_ylabel('TB (K) C090')
for x in range(5):
    axes[0,x].set_xticklabels([])
plt.show()

#fig.savefig('../Figs/Fig8_SS.png', bbox_inches = 'tight')#, pad_inches = 0)

In [ ]:
for k in obs_87['plateau'].keys():
    print(k, np.median(obs_87['slopes'][k]), np.median(obs_87['valley'][k]), np.median(obs_87['plateau'][k]))

In [ ]:
for k in obs_90['plateau'].keys():
    print(k, np.median(obs_90['slopes'][k]), np.median(obs_90['valley'][k]), np.median(obs_90['plateau'][k]))

In [ ]:
for k in sims_87['plateau'].keys():
    print(k, np.median(sims_87['slopes'][k]), np.median(sims_87['valley'][k]), np.median(sims_87['plateau'][k]))

In [ ]:
for k in sims_90SS['plateau'].keys():
    print(k, np.median(sims_90SS['slopes'][k]), np.median(sims_90SS['valley'][k]), np.median(sims_90SS['plateau'][k]))

In [ ]:
obs_90, sims_90 = group_topography(obs_c090, smrt_c090)

In [ ]:
# Table in paper
print ('channel', 'slopes', 'valley', 'plateau')
for k in sims_90SS['plateau'].keys():
    print(k, 
          np.round(np.median(sims_90SS['slopes'][k]) - np.median(sims_90['slopes'][k]),1), 
          np.round(np.median(sims_90SS['valley'][k]) - np.median(sims_90['valley'][k]),1), 
          np.round(np.median(sims_90SS['plateau'][k]) - np.median(sims_90['plateau'][k]),1))